<h1 style="text-align: center; color: blue">Mesh Smoothing</h1>

Mesh smoothing is concerned with the design and computation of smooth functions $f : S \rightarrow \mathbf{R}^d$ on a triangle mesh. The function $f$ can flexibly be chosen to describe, for instance, vertex positions, texture coordinates, or vertex displacements. In this notebook, I will show how to perform mesh smoothing using several techniques based on denoising and fairing.

Denoising is used to remove high-frequency noise from the function $f$. In most cases, $f$ denotes the vertex positions, which might be corrupted by high frequency noise due to a physical scanning process. Removing the noise (the high frequencies) and keeping the overall shape (the low frequencies) requires generalizing the concepts of frequencies and low-pass filters to functions living on discrete triangle meshes. 

Mesh fairing does not just slightly smooth the function $f$ in order to remove the high frequency noise. It also smooths the function as much as possible in order to obtain, e.g., an as-smooth- as-possible surface patch or an as-smooth-as-possible shape deformation.

[Polygon Mesh Processing by M. Botsch et al]

In [2]:
# importing all libraries
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp
import igl

In [157]:
# Read the mesh data
v, f = igl.read_triangle_mesh("data/bunny.off")

n = igl.per_vertex_normals(v, f)
c = np.linalg.norm(n, axis=1)

mp.plot(v,f,n,shading={"wireframe":True, "width":400, "height":400})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

<h3 style="color: #9933ff;"> Smoothing By Diffusion Flow </h3>

Diffusion flow is modeled by the diffusion equation 

$$\frac{\partial f(x,t)}{\partial t} = \lambda \nabla f(x,t)$$

For our smoothing purposes, $f$ here means the vertex position. We can employ the diffusion equation to smooth an arbitrary function $f : S \rightarrow \mathbb{R}$ on a manifold surface $S$, simply by replacing the regular Laplace operator by the manifold Laplace-Beltrami, (i.e. $\nabla p = Lp$), where $L$ is the Laplace-Beltrami operator. Thus by discretizing the time partial derivative using forward-euler, we get:

$$\hat{p} = p + \lambda hL p$$

<h4> Explicit Smoothing </h4>

In [162]:
# Smoothing using cotangent Laplacian
L = igl.cotmatrix(v,f)

alpha = 0.02
iterations = 10000
v_smooth = v.copy()

# Explicit Smoothing
for i in range(iterations):
    v_smooth = v_smooth + alpha * L.dot(v_smooth)
    # Taubin method to inflate the surface
    v_smooth = v_smooth - alpha * L.dot(v_smooth)

v_smooth[:,0] += 0.2
p = mp.plot(v,f,n)
p.add_mesh(v_smooth, f, n)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

1

<h4>Implicit Smoothing</h4>

In [163]:
alpha = 1.5
L = igl.cotmatrix(v,f)

A1 = (np.eye(v.shape[0]) - alpha* L)
v_smooth = v.copy()

v_smooth[:,0] = sp.linalg.spsolve(A1, v[:,0]) 
v_smooth[:,1] = sp.linalg.spsolve(A1, v[:,1]) 
v_smooth[:,2] = sp.linalg.spsolve(A1, v[:,2]) 


v_smooth[:,0] += 0.2
q = mp.plot(v,f,n)
q.add_mesh(v_smooth,f,n)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

1

<h3 style="color: #9933ff;"> Smoothing as Optimization </h3>

The goal is to calculate

$$
\nabla_M \tilde{p} = -2Hn = 0
$$
with $H = 0$

However, this will give us the trivial solution. One different approach is to regularize

$$
\min_{\tilde{p}} \int_M || \nabla_M \tilde{p}||^2 + \omega || \tilde{p} - p||^2
$$

Now using the Laplacian-Beltrami operator we have $\nabla_M p = L p$, where $L = M^{-1} L_w$. Threfore,

$$
E = (L\tilde{p})^T M (L \tilde{p}) + \omega (\tilde{p} - p)^T M (\tilde{p} - p) \\
\frac{\partial{E}}{\partial \tilde{p}} = L^T M L + \omega M (\tilde{p} - p) = 0 \\
(L^TML + \omega M) \tilde{p} = \omega M p \\
(L_w^T M^{-1} L_w + \omega M) \tilde{p} = \omega M p \\
\tilde{p} = (L_w^T M^{-1} L_w + \omega M)^{-1} \omega M p
$$

In [207]:
L = igl.cotmatrix(v,f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
Minv = sp.diags(1. / M.diagonal())
w = 0.5

A = L.T @ M @ L + w * M
b = w * (M.dot(v))

v_smooth = sp.linalg.spsolve(A,b)
v_smooth[:,0] += 0.2

r = mp.plot(v,f,n)
r.add_mesh(v_smooth,f,n)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

1